In [2]:
import backoff
import requests
import os
import httpx
import openai
from openai import OpenAI

os.environ['BASE_URL'] = "https://svip.xty.app/v1"
os.environ['API_KEY'] = "sk-r0WeYOdkMjzYdnSxEcC8B931Aa904e4bBaCcAc2a57D803F1"
# os.environ['BASE_URL'] = 'https://api.siliconflow.cn/v1'
# os.environ['API_KEY'] = 'sk-gkdahtyanpeqrloadhiqbjarcmzfqlbrpjzhummgqxnedhjw'
client = openai.OpenAI(
    base_url=os.getenv("BASE_URL"),
    api_key=os.getenv("API_KEY"),
    http_client=httpx.Client(
        base_url=os.getenv("BASE_URL"),
        follow_redirects=True,
    ),
)

MAX_TRIES=10
MAX_TIME=10
@backoff.on_exception(backoff.expo, openai.OpenAIError, max_tries=MAX_TRIES, max_time=MAX_TIME, raise_on_giveup=True)
def completion_with_backoff(**kargs):
    return client.chat.completions.create(
        **kargs
    )
def query_model(messages, temperature=0.7, max_tokens=1024):
    messages=[
        {"role": "system", "content": ""},
        {"role": "user", "content": messages},
    ]
    for i in range(3):
        # print("length:",len(str(messages)))
        response = completion_with_backoff(
            model="deepseek-v3-250324",
            messages=messages,
            temperature=temperature,
            **{"max_tokens": max_tokens},
            stream=True  # 启用流式输出
        )
        # 逐步接收并处理响应
        
        
        collected_chunks = []
        for chunk in response:
            if not chunk.choices:
                continue
            if chunk.choices[0].delta.content:
                collected_chunks.append(chunk.choices[0].delta.content)

        # 组合输出片段并返回
        content= ''.join(collected_chunks)
        # content = response.choices[0].message.content
        # print("res",content)
        if content=="":
            continue
        return content
    return "No Response."

In [3]:
import json
import os
def load_json(path):
    with open(path,"r") as f:
        js=json.load(f)
    return js

def save_json(path,dict):
    with open(path,"w",encoding="utf-8") as f:
        json.dump(dict,f,ensure_ascii=False)
    
files=os.listdir("../data/data0417")
print(len(files))

200


In [4]:
articles_per_crime=load_json("articles_fine2.json")
print(len(articles_per_crime))

400


In [5]:
law_repository=load_json("../resource/law.json")
print(len(law_repository.keys()))

55348


In [6]:
pwd='../data_generation'
cases=[]
with open(os.path.join(pwd,'data.jsonl')) as f:
    for line in f:
        case = json.loads(line)
        cases.append(case)
print("cases loaded.")

cases loaded.


In [7]:
# from tqdm import tqdm
# merge_dict=[]
# seg=load_json("./seg_dict.json")
# for file in tqdm(files):
#    id=int(file.split("_")[-1])
#    segment=seg[str(id)]
#    js=load_json(os.path.join("../data/data0417",file,"data_anonymized.json"))
#    term=js["term"]
   
#    fine=js["fine"]
#    merge_dict.append({"id":id,"seg":segment,"term":term,"fine":fine})
# merge_dict=sorted(merge_dict, key=lambda x: float(x['id']))
# save_json("./merge_dict.json",merge_dict)

In [8]:
articles_per_crime=load_json("articles_fine2.json")
print(len(articles_per_crime))

400


In [9]:
# from tqdm import tqdm
# border=[]
# seg=load_json("./seg_dict.json")
# for file in tqdm(files):
#     id=int(file.split("_")[-1])
   
#     articles=articles_per_crime[id]
#     assert(articles['id']==id)
#     arts=""
#     for art in articles["crimes"]:
#         art_="第"+art.split("第")[1]
#         art_crime="中华人民共和国刑法"+art_
#         art_content=law_repository[art_crime]
#         arts+="《"+art_crime+"》:"+art_content+"\n\n"
#     # prosecution=js['prosecution_statement']
#     segment=seg[str(id)]
#     js=load_json(os.path.join("../data/data0417",file,"data_anonymized.json"))
#     term=js["term"]
#     fine=js["fine"]
#     case=cases[id]
#     info=f"""
#         【查明事实、法律认定与判决】：{case['fact']+case['reason']+case['result']}
#         【最终判罚月数】：{term}
#         【相关法条】：{arts}
#         """
    
#     if segment=="无罪":    
#         border.append({"id":id,"seg":segment,"term":term,"fine":fine,"info":info})
#     else:
#         ss=segment.split("#")
#         if float(ss[0])==term or float(ss[1])==term:
#             border.append({"id":id,"seg":segment,"term":term,"fine":fine,"info":info})
# border=sorted(border, key=lambda x: float(x['id']))
# border=border[0:]
# print(len(border))
# save_json("./border.json",border)

In [10]:
# from tqdm import tqdm
# seg_dict={}

# for file in tqdm(files):
#     id=int(file.split("_")[-1])
#     js=load_json(os.path.join("../data/data0417",file,"data_anonymized.json"))
#     term=js["term"]
#     articles=articles_per_crime[id]
#     assert(articles['id']==id)
#     arts=""
#     for art in articles["crimes"]:
#         art_="第"+art.split("第")[1]
#         art_crime="中华人民共和国刑法"+art_
#         art_content=law_repository[art_crime]
#         arts+="《"+art_crime+"》:"+art_content+"\n\n"
#     # prosecution=js['prosecution_statement']
#     case=cases[id]
#     seg=query_model(
#         f"""
#         你是刑法学专家。在《中华人民共和国刑法》规定中，每个判罚有一个参考的判罚区间。
#         现在，我希望你能够通过【起诉书】，【查明事实、法律认定与判决】，【相关法条】寻找出这个判罚区间。
#         你应当返回这个区间的左右端点，左右端点都换算为月为单位。
#         左右端点用阿拉伯数字表示，之间用#分隔。
#         如果是上不封顶的，比如十年以上，右端点为99999
        
        
#         例如：
#         【查明事实、法律认定与判决】：被告人刘某甲积极退赔部分经济损失，取得被害人谅解，可酌情从轻处罚。建议判处被告人刘某甲有期徒刑一年十个月，并处罚金人民币三千元。
#         【最终判罚月数】：22
#         【相关法条】：盗窃公私财物，数额较大的，或者多次盗窃、入户盗窃、携带凶器盗窃、扒窃的，处三年以下有期徒刑、拘役或者管制，并处或者单处罚金；数额巨大或者有其他严重情节的，处三年以上十年以下有期徒刑，并处罚金；数额特别巨大或者有其他特别严重情节的，处十年以上有期徒刑或者无期徒刑，并处罚金或者没收财产。
        
#         （可以发现，相关法条中有3段不同的区间，而最后判了22个月，这属于三年以下有期徒刑区间。
#         所以左端点是0，右端点是36（三年转化为月是36个月））
#         你应当返回：0#36
        
#         下面，请按照格式返回下面案件中的左右端点。
#         【查明事实、法律认定与判决】：{case['fact']+case['reason']+case['result']}
#         【最终判罚月数】：{term}
#         【相关法条】：{arts}
        
#         注意！
#         1.你必须严格参考相关法条中的内容。
#         2.刑法中所称以上、以下、以内，包括本数。对于边界情况，需要你自行根据判罚轻重决定这个区间是往上还是往下。
#           例如，判了三年，但是刑法中要么是三年以下，要么是三年以上十年以下。假如，你通过分析发现，这已经是从轻处罚后的结果，那么说明适用范围应该是三年以上十年以下。（如果是从重处罚的结果，那么实际适用范围应该是三年以下）
#         3.请严格按照要求的格式返回，不要说多余的话。
#         4.如果是上不封顶的，比如十年以上，右端点为99999
#         5.对于无法确定的，请直接输出‘我不确定’！
#         """
#     )
#     print(id,seg)
#     # seg=seg.split("#")
#     seg_dict.update({id:seg})
#     # save_json("./seg_dict.json",seg_dict)

In [11]:
from tqdm import tqdm
fine_list=[]

seg=load_json("./seg_dict.json")
for file in tqdm(files):
    id=int(file.split("_")[-1])
    # js=load_json(os.path.join("../data/data0417",file,"data_anonymized.json"))
    # term=js["term"]
    # print(str(id))
    articles=articles_per_crime[id]
    assert(articles['id']==id)
    segment=seg[str(id)]
    arts=""
    for art in articles["crimes"]:
        art_="第"+art.split("第")[1]
        art_crime="中华人民共和国刑法"+art_
        art_content=law_repository[art_crime]
        arts+="《"+art_crime+"》:"+art_content+"\n\n"
    # prosecution=js['prosecution_statement']
    case=cases[id]
    
    if segment=="无罪":
        seg_str="无罪"
    else:
        seg_str=segment.split("#")
        seg_str=f"{seg_str[0]}个月到{seg_str[1]}个月"
    
    fine=query_model(
        f"""
        你是刑法学专家。在《中华人民共和国刑法》规定中，每个判罚有一个参考的判罚区间。
        现在，给你一个法条，其中有多个判刑区间，例如xxxx（情节），处三年以下；xxxxx（情节），处三年以上七年以下
        同时，每一个区间也可能有罚金说明，你要把罚金说明的部分提出来。
        
        提供法条内容，案件事实，判刑区间。
        
        例如：
        【查明事实、法律认定与判决】：被告人刘某甲积极退赔部分经济损失，取得被害人谅解，可酌情从轻处罚。建议判处被告人刘某甲有期徒刑一年十个月，并处罚金人民币三千元。
        【最终判罚月数】：0个月到36个月
        【相关法条】：盗窃公私财物，数额较大的，或者多次盗窃、入户盗窃、携带凶器盗窃、扒窃的，处三年以下有期徒刑、拘役或者管制，并处或者单处罚金；数额巨大或者有其他严重情节的，处三年以上十年以下有期徒刑，并处罚金；数额特别巨大或者有其他特别严重情节的，处十年以上有期徒刑或者无期徒刑，并处罚金或者没收财产。
        
        （可以发现，对应区间里面“处三年以下有期徒刑”的条目，所以罚金的说明是“并处或者单处罚金”
        你的返回是：并处或者单处罚金
        
        
        输入：
        【查明事实、法律认定与判决】：{case['fact']+case['reason']+case['result']}
        【最终判罚月数】：{seg_str}
        【相关法条】：{arts}
        给出你提取的罚金内容。
        
        注意！
        1.你必须严格参考相关法条中的内容。
        2.请严格按照要求的格式返回，不要说多余的话。
        3.对于无法确定的，请直接输出‘我不确定’！
        """
    )
    print(id,fine)
    # seg=seg.split("#")
    fine_list.append({"id":id,"seg":seg_str,"fine":fine,"arts":arts})
    save_json("./fine_list.json",fine_list)

  0%|          | 1/200 [00:04<13:49,  4.17s/it]

210 并处罚金或者没收财产


  1%|          | 2/200 [00:07<12:37,  3.83s/it]

384 并处罚金


  2%|▏         | 3/200 [00:09<09:54,  3.02s/it]

...

220 并处罚金


100%|█████████▉| 199/200 [08:45<00:02,  2.74s/it]

350 我不确定


100%|██████████| 200/200 [08:51<00:00,  2.66s/it]

41 并处罚金


In [25]:
js=load_json("./fine_list.json")
out=[]
for file in tqdm(files):
    id=int(file.split("_")[-1])
    dic=load_json(os.path.join("../data/data0417",file,"data_anonymized.json"))
    fine=dic["fine"]
    
    for tmp in js:
        if tmp['id']==id:
            tmp.update({"amount":fine})
            out.append(tmp) 
            break  
out=sorted(out,key=lambda x: int(x['id']))    
save_json("./fine_list.json",out)

100%|██████████| 200/200 [00:00<00:00, 7425.98it/s]


In [26]:
js=load_json("./fine_list.json")
save_json("./tmp_list.json",js[:50])

In [27]:
tmp=[i*10+j for i in range(40) for j in range(5)]

lst=[{"id":i,"fine":""} for i in tmp]
lst
# save_json("./fine_seg.json",lst)

old_fine_seg=load_json("./fine_seg.json")

fine_list=load_json("./fine_list.json")

for dict in fine_list:
    if dict["amount"]==0:
        for j in range(len(old_fine_seg)):
            if old_fine_seg[j]["id"]==dict["id"]:
                old_fine_seg[j]["fine"]="0#0"
save_json("./fine_seg.json",old_fine_seg)

In [29]:
crime_name=['危险驾驶罪', '盗窃罪', '故意伤害罪', '交通肇事罪', '走私、贩卖、运输、制造毒品罪', '诈骗罪', '寻衅滋事罪', '容留他人吸毒罪', '开设赌场罪', '妨害公务罪', '抢劫罪', '非法持有、私藏枪支、弹药罪', '非法拘禁罪', '非法持有毒品罪', '信用卡诈骗罪', '赌博罪', '滥伐林木罪', '故意毁坏财物罪', '非法经营罪', '合同诈骗罪', '聚众斗殴罪', '受贿罪', '敲诈勒索罪', '非法吸收公众存款罪', '贪污罪', '引诱、容留、介绍卖淫罪', '故意杀人罪', '抢夺罪', '职务侵占罪', '非法制造、买卖、运输、邮寄、储存枪支、弹药、爆炸物罪', '强奸罪', '非法占用农用地罪', '拒不执行判决、裁定罪', '虚开增值税专用发票、用于骗取出口退税、抵扣税款发票罪', '生产、销售、提供假药罪', '过失致人死亡罪', '伪造、变造、买卖国家机关公文、证件、印章罪', '生产、销售有毒、有害食品罪', '行贿罪', '非法捕捞水产品罪', '盗伐林木罪', '窝藏、包庇罪', '放火罪', '失火罪', '组织、领导传销活动罪', '污染环境罪', '挪用资金罪', '挪用公款罪', '非法种植毒品原植物罪', '拒不支付劳动报酬罪']
print(crime_name[:40])

['危险驾驶罪', '盗窃罪', '故意伤害罪', '交通肇事罪', '走私、贩卖、运输、制造毒品罪', '诈骗罪', '寻衅滋事罪', '容留他人吸毒罪', '开设赌场罪', '妨害公务罪', '抢劫罪', '非法持有、私藏枪支、弹药罪', '非法拘禁罪', '非法持有毒品罪', '信用卡诈骗罪', '赌博罪', '滥伐林木罪', '故意毁坏财物罪', '非法经营罪', '合同诈骗罪', '聚众斗殴罪', '受贿罪', '敲诈勒索罪', '非法吸收公众存款罪', '贪污罪', '引诱、容留、介绍卖淫罪', '故意杀人罪', '抢夺罪', '职务侵占罪', '非法制造、买卖、运输、邮寄、储存枪支、弹药、爆炸物罪', '强奸罪', '非法占用农用地罪', '拒不执行判决、裁定罪', '虚开增值税专用发票、用于骗取出口退税、抵扣税款发票罪', '生产、销售、提供假药罪', '过失致人死亡罪', '伪造、变造、买卖国家机关公文、证件、印章罪', '生产、销售有毒、有害食品罪', '行贿罪', '非法捕捞水产品罪']
